In [1]:
import random

import cv2
import numpy as np

Download images from directory.

In [2]:
from image import read_image, show_image

In [3]:
images, image_names = read_image("InputImages")

In [4]:
image = images[0]

# Stratification

Get lines from the image.

In [5]:
from line import get_lines, draw_line

In [6]:
lines = get_lines(image)

Plot lines.

In [ ]:
ouput_image = image.copy()
# Getting the lines form the image
for line in lines:
    r = random.randint(0, 256)
    g = random.randint(0, 256)
    b = random.randint(0, 256)
    draw_line(ouput_image, line, (r, g, b))
show_image("OutputImage", ouput_image)

Cluster lines to find vanishing points.

In [7]:
from clustering import cluster_lines

In [8]:
n_clusters = 3

In [9]:
clusters = cluster_lines(lines, n_clusters=n_clusters)
clusters

defaultdict(list,
            {0: [array([[-240.       ,    2.0769417]], dtype=float32),
              array([[-177.       ,    2.1816616]], dtype=float32),
              array([[-244.       ,    2.0769417]], dtype=float32),
              array([[-216.       ,    2.0943952]], dtype=float32),
              array([[-188.       ,    2.1991148]], dtype=float32),
              array([[-143.       ,    2.2165682]], dtype=float32),
              array([[-128.       ,    2.2863812]], dtype=float32),
              array([[-229.       ,    2.0594885]], dtype=float32),
              array([[-226.       ,    2.1118484]], dtype=float32),
              array([[-159.       ,    2.2340214]], dtype=float32),
              array([[-256.       ,    2.0071287]], dtype=float32),
              array([[-205.      ,    2.146755]], dtype=float32)],
             2: [array([[-44.       ,   2.4085543]], dtype=float32),
              array([[3.       , 2.4783676]], dtype=float32),
              array([[-6.       ,

Show image with clustered lines and find vanishing points.

In [10]:
from homography import get_vanishing_point, get_distant_points

In [11]:
vanishing_points = [get_vanishing_point(clusters[c]) for c in clusters]
vanishing_points

[array([  229.23634, -1241.6215 ], dtype=float32),
 array([-607.8082 , -281.19775], dtype=float32),
 array([2829.3293, -731.7135], dtype=float32)]

In [ ]:
colors = [(random.randint(0, 256), random.randint(0, 256), random.randint(0, 256)) for _ in range(n_clusters)]

output_image = image.copy()
for c in clusters:
    vp = vanishing_points[c]
    for i, line in enumerate(clusters[c]):
        draw_line(output_image, line, colors[c])
    cv2.circle(output_image, (int(vp[0]), int(vp[1])), 10, colors[c], -1)

show_image("OutputImage", output_image)  

In [12]:
two_points, _ = get_distant_points(vanishing_points)
two_points

[array([-607.8082 , -281.19775], dtype=float32),
 array([2829.3293, -731.7135], dtype=float32)]

Perform stratification.

In [13]:
H1 = np.eye(3)
H1[2] = np.cross([*two_points[0], 1], [*two_points[1], 1])
H1[2] /= H1[2][2]
print(H1)

[[1.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00]
 [3.63218814e-04 2.77111963e-03 1.00000000e+00]]


In [14]:
new_image = cv2.warpPerspective(image, H1, (image.shape[1], image.shape[0]))
show_image("OutputImage", new_image)

# Metric rectification

Get lines from transformed images.

In [15]:
new_lines = get_lines(new_image, threshold=80, lines=10, srn=15)
len(lines)

21

In [ ]:
output_image = new_image.copy()
for line in new_lines:
    r = random.randint(0, 256)
    g = random.randint(0, 256)
    b = random.randint(0, 256)
    draw_line(output_image, line, (r, g, b))
show_image("OutputImage", output_image)

In [16]:
new_clusters = cluster_lines(new_lines, n_clusters=n_clusters)
new_clusters

defaultdict(list,
            {1: [array([[324.       ,   1.1170107]], dtype=float32),
              array([[330.       ,   1.0995574]], dtype=float32),
              array([[313.      ,   1.134464]], dtype=float32),
              array([[332.       ,   1.0995574]], dtype=float32),
              array([[316.      ,   1.134464]], dtype=float32)],
             0: [array([[234.       ,   1.4835298]], dtype=float32),
              array([[55.       ,  1.4486233]], dtype=float32),
              array([[120.     ,   1.43117]], dtype=float32),
              array([[124.       ,   1.4137167]], dtype=float32),
              array([[201.       ,   1.4137167]], dtype=float32),
              array([[113.       ,   1.4486233]], dtype=float32),
              array([[133.     ,   1.43117]], dtype=float32),
              array([[107.       ,   1.4137167]], dtype=float32),
              array([[53.       ,  1.4660766]], dtype=float32),
              array([[101.       ,   1.4835298]], dtype=float32),
 

In [21]:
colors = [(random.randint(0, 256), random.randint(0, 256), random.randint(0, 256)) for _ in range(n_clusters)]

output_image = new_image.copy()
for c in new_clusters:
    for line in new_clusters[c]:
        draw_line(output_image, line, colors[c])

show_image("OutputImage", output_image)  

Find orthogonal lines.

In [22]:
from line import find_orthogonal_lines

In [27]:
len(new_clusters)

3

In [25]:
orthogonal_lines = find_orthogonal_lines(new_clusters)
orthogonal_lines

[[array([[30.       ,  1.6231562]], dtype=float32),
  array([[330.       ,   1.0995574]], dtype=float32)],
 [array([[204.       ,   1.2915436]], dtype=float32),
  array([[-97.       ,   2.5481806]], dtype=float32)],
 [array([[330.       ,   1.0995574]], dtype=float32),
  array([[-97.       ,   2.5481806]], dtype=float32)]]

In [26]:
colors = [(random.randint(0, 256), random.randint(0, 256), random.randint(0, 256)) for _ in range(len(orthogonal_lines))]

output_image = new_image.copy()
for i, lines in enumerate(orthogonal_lines):
    for line in lines:
        draw_line(output_image, line, colors[i])

show_image("OutputImage", output_image)

Make linear equation from 3 orthogonal lines.

In [30]:
from line import get_homogeneous_coordinates

Suppose that $s_{12}$ = $s_{21}$= 1

In [50]:
A = []
b = []
for pair in orthogonal_lines[:2]:
    l, m = get_homogeneous_coordinates(pair[0]), get_homogeneous_coordinates(pair[1])
    A.append([l[0] * m[0], l[1] * m[1]])
    b.append(-(l[0] * m[1] + l[1] * m[0]))

In [51]:
s = np.linalg.solve(A, b)
s

array([-4.148778, -0.567903], dtype=float32)

In [53]:
C = np.array([
    [s[0], 1, 0],
    [1, s[1], 0],
    [0, 0, 1],
])

In [54]:
u, s, vh = np.linalg.svd(C, full_matrices=True)

In [57]:
u

array([[-0.96774359,  0.        ,  0.2519372 ],
       [ 0.2519372 ,  0.        ,  0.96774359],
       [ 0.        ,  1.        ,  0.        ]])

In [58]:
s

array([4.40911262, 1.        , 0.30756832])

In [59]:
vh

array([[ 0.96774359, -0.2519372 ,  0.        ],
       [ 0.        ,  0.        ,  1.        ],
       [-0.2519372 , -0.96774359, -0.        ]])

Then metric rectification is u matrix.

In [61]:
H2 = u

In [62]:
output_image = cv2.warpPerspective(new_image, H2, (new_image.shape[1], new_image.shape[0]))
show_image("OutputImage", output_image)